In [ ]:
#!kaggle competitions download - c jpx-tokyo-stock-exchange-prediction


In [ ]:
#!unzip - d ./data jpx-tokyo-stock-exchange-prediction.zip


## Some observation about data
### stock_price
* open high low close volume --- daily frequency   
* securitiescdoe --- stock id
* rowid --- row num --- trade date+security code
* adjustmentfactor --- to calculate theoretical price/volume when split/reverse-split happens (NOT including dividend/allotment of shares/)
    * split https://baike.baidu.com/item/%E8%82%A1%E7%A5%A8%E5%88%86%E5%89%B2/ https://en.wikipedia.org/wiki/Stock_split 1 stock -> 2 stock so price half
    * reverse split https://baike.baidu.com/item/%E5%8F%8D%E5%90%91%E5%88%86%E5%89%B2 https://en.wikipedia.org/wiki/Reverse_stock_split
* SupervisionFlag --- under supervision / will leave the market 
* ExpectedDividend --- Expected dividend value for ex-right date. This value is recorded 2 business days before ex-dividend date. --- may be temporarily ignored --- I don't know how to use hhh...
    * 原本上涨的好的在除权除息之后容易被看好，“填权”；反之容易受到看空，但都不一定；除权除息本身是中性行为
* target

### trades 
* Aggregated summary of trading volumes from the previous business week.
    * three kinds of markets reported at next week Thur. data about last week (usually 5 days)
    * 好吧英语太差了 这一段拿中文写 看起来有三种市场 我不确定它们对于一只给定的股票 是否是互斥的，当然是不是都能做，只要知道它属于哪个类别就可以试试，比如对于第t-1周的数据，第t周要报告，那么在t周时，对给定的股票，可以给每天的数据都加上这样的特征（五天都一样），认为上一周的交易额会影响股价
    或者为了避免五天都一样，可以周一取前n周平均？周二取n-1平均，周五取之前n-4周平均，---- 历史对现在的影响
    ---- 大概是一个基本面的因子 市场之类

### stock_list
* section/products newmarketsegmet ref https://baijiahao.baidu.com/s?id=1687334861307709525&wfr=spider&for=pc https://baijiahao.baidu.com/s?id=1705781744998764716&wfr=spider&for=pc 大概就是A股的什么主板 中小板 创业板...
* effective date??
* SecuritiesCode --- stock id --- mapping to stock price
* 33SectorCode/33SectorName/17SectorCode/17SectorName -stock kind by industry (industry code)
* NewIndexSeriesSizeCode/NewIndexSeriesSize --- index (股指)
* tradedate/close price
* Issued shares
* Universe0 --- a flag of prediction target universe (top 2000 stocks by market capitalization) --- should have 2000 items (2000 true)

### finacials
* Results from quarterly earnings reports.
* I don't know how to use...

### secondary_stock_prices
* The core dataset contains on the 2,000 most commonly traded equities but many less liquid securities are also traded on the Tokyo market. This file contains data for those securities, which aren't scored but may be of interest for assessing the market as a whole.
* flaged FALSE in column universe0 in stock_list
* assess the market as a whole
* I don't know how to use...

### options
* Data on the status of a variety of options based on the broader market. **Many options include implicit predictions of the future price of the stock market** and so may be of interest even though the options are not scored directly.
* important I think,  price of options can reflect returns and risk...
* we need to/should know underlying securities
* 事实上不太好用，大概的思路是期权的定价可以同时反应标的的收益和风险，虽然理论上标的可以是任意东西，但从optioncode来看一般是股指或者金银等贵金属汽油之类，要求预测的stock是个股，唯一可能尝试这个个股是否在某个股指中，然后做一个映射，但查看appendix来看，topix的股指和个股用的股指似乎不是一套，最多有一个topix core30 index是一致的，因此这个特征可能不是很好用。
* So actually  I don't know how to use...

### To make it, we really need someone familiar with economics, finance and stocks orz...

## Rethink about the target/task
这一段拿中文写了 
* https://www.kaggle.com/code/smeitoma/jpx-competition-metric-definition   
* 假设比赛的评分期是三个月，从上面那个链接看，第五条，对三个月中每一个交易日，不妨记为$t$，都会计算这天的“daily spread return”，记为$R_t$, 在得到三个月中每个交易日的$R_t$之后，求三个月的夏普，这个值作为score
* 对于夏普来说，这个日频的$R_t$，值越大（即收益越高），标准差越小（即风险越小），夏普越高。 --- $R_t$构成一个时间序列
* 关键是计算每个交易日的$R_t$，这个$R_t$，是由两篮子股票构成的，第一篮子叫top200，第二篮子叫bottom200，我们在t+1天做多top200，做空bottom200，在t+2天卖出，把一篮子股票的加权等价为一只股票，这两相当于手里有两只股票，一个叫top200，一个叫bottom200，对于top200，希望t+1到t+2的收盘价变化率尽可能大（又正又大），即t+2收盘价很高，t+1收盘价很低，对bottom200，希望反过来，希望这个收盘价变化率越小越好（又小又负）（亏的越多越开心，因为做空）（即t+2收盘价很低，t+1收盘价很高）。这是对于一个交易日的结果。
* 此外要控制风险，比较简单的一个方法是，相邻两天，或某个窗口内的$R_t$，极差不要过大，这也与某个discussion吻合，不要一味追求高的$R_t$，同样要控制风险。
* 问：用supervised learning 应该选什么做y？--- 注意这不是一个时间序列的模型，我们只能给出一个预测值，如果想到closing price的水平，除非训两个模型，一个预测t+1的，一个预测t+2的，这样做或许有些复杂。
* 先这样考虑，先不控制风险，只追求高收益，可以直接把每只股票在t天的，从t+1到t+2的closing price变化率作为y（即raw training set中的target），把y_pred预测值排序，找到top200和bottom200，自然可以得到这天的$R_t$。发现只要y预测的足够好（指与真实市场一致，事实上只有到t+2天结束，我们才能知道t天的y_true），一定可以得到高收益。
* 如何控制风险 --- 可能会有别的度量，这个不是我专业hhhh，这里就用夏普，我们换一个思路，我们假设y，也就是变化率预测的足够好，那么由它求出的个股的夏普一定也是准的（与真实数据相比），那么用这个求出的夏普来rank，一定可以综合反映收益和风险。--- 因此 We should predict the change rate, then calculate sharpe ratio of a single stock, finally sort(rank) by the calculated(predicted) sharpe ratio to get top200 and bottom200.
* 下一个问题是 sharpe ratio 需要一个window，选多宽？这个问题可能要仔细思考 --- 我目前没有什么好的答案hhh
* 这样问题集中在，如何获得好的预测值y_pred，上述predicted Sharpe ratio是得到预测值之后的事情，可以先不考虑。事实上之前提到做两个模型一个预测t+1的，一个预测t+2的closing price或许会比直接预测收益率要好，当然这是我的intuition，大概源于rate并不是直接的结果，是一个规则计算出的，用model去预测rate或许还要把这个规则也学出来？（我的猜测，加入model实际上预测price的能力很强...
* 以下先学rate...

## Feature engineering IDEA   
* 为了设计特征，问：什么会影响t+1到t+2的rate？可以整体分为两个方面，1. 当天的数据 在t天时，可以获得的信息 2. 历史信息，过往若干天的信息
* 对第一个方面，actually cross section，除了自己的数据（open/close...），还可以有行业，股指，大盘，板块，甚至星期，月，季度
    * 行业 星期 月 季度 或许可以用one-hot编码，属于什么股指也可以编码一下或许
        * 看起来可以把星期和月用ordinal encoding https://machinelearningmastery.com/one-hot-encoding-for-categorical-data/ 
    * 股指、大盘、板块的交易信息（open/close）
    * 此外可以找一下只需要当天数据就能获取的factor
* second part --- 我把它分为直接时序数据和间接时序数据
    * 直接是指lag feature 用t-n~t-1的rate，open，close...
    * 间接归到factors from quantitative finance，eg EMA MACD volatility...
* <font style='color: red;'> **所以换一种分类方式吧** </font>
    * raw data on day t + some simple transformation like log/++-- + coded extra informaiton like industry/index/market/time...
    * quantitative finance factors
    * lag features
* 一点思考，关于为什么需要量化因子，如果我们能先验的知道某个因子很有效，那么model或多或少要把这个因子是如何用raw data构造的**规则**学出来，这或许会增加训练难度，不如直接构造好塞给model，让model去挖掘更加未知的信息

## Some questions
* I'm not sure what 'SupervisionFlag' means and how it will influence model (influence test)
* 事实上是有一些诡异的hhh，有2000支股票，只用一个模型，相当于一招鲜吃遍天，一个模型把大家都预测对了，但我认为这是比较难的，因为A股票的pattern在B股票上不一定好用，这样B股票的数据就变成噪音了，除非做一个2000维的one-hot，把其它都silent。/////// 最理想的应该是，2000个model，每个只负责一只股票，这属于术业有专攻，理论上应该会比一招鲜预测的好，但是看起来不太现实
* 另一个困难的问题是cv怎么做hhh
    * 时间序列的cv原则上必须按照时间顺序，从前到后 --- 非均匀划分
    * 如果用一招鲜的模型，对个股的特征肯定还是要用个股的数据提，相当于先拆出来个股，对每个个股提完特征，再拼回去，再按时间划分，然后再把时间都去掉，最后做成训练和验证
* adjustmentfactor 可能还真的要调整一下理论的量价 https://www.kaggle.com/code/vweimer/jpx-adjusted-close-and-volume-and-features/notebook?scriptVersionId=96081798
* 为了对行业板块之类的信息编码，需要全市场的数据，因此在针对个股提特征之前，应该先用groupby把全市场的数据过一遍然后编码   
* 可以大致把流程分为
    1. <font style='color: red;'> **Adjust price and volume** </font> 所有数据按股票代码分组，得到每只股票在时间范围内的数据，按adjustmentfactor调整每只股票的量价，得到理论值，combine
    2. <font style='color: red;'> **Encode week, month, market...** </font> 编码星期，月份，板块，可以直接处理全部数据
    3. <font style='color: red;'> **Encode industry, index** </font>所有数据按Date分组，得到每一天的全市场数据，对全市场数据，再按三个方面（33sector...）分组，在每个方面下计算同组量价，这一步相当于对33sector等categorical var. 做一个基于统计的编码，combine
    4. <font style='color: red;'> **FE of each stocks** </font> 所有数据按股票代码分组，得到个股时序交易数据，对个股提特征，xxxx方面，combine
    5. <font style='color: red;'> **split training and validation** </font>按照时序的惯例划分，这一步有一些诡异其实hhh，正常的cv应该在最开始做，然后对training和validation做一模一样的事情，得到特征。此处是为了方便，基于这样的想法：
        * 假设需要预测第t天的（validation和test可以同论），我手里会有t当天的数据，这是naive的，直接可以获得；第t-1天及之前的数据，其实是有的，在training set里面（当然要假设整个时间是连续的）（但想获得可能有一些难度，这个之后会详述）。对于只需要day t数据的特征，可以很容易的得到，但是，无论是lag features or some quantitative factors，都会用到t-1及之前的数据。换言之，为了得到第t天的特征，我其实需要一个从t-n到t的矩阵，作为输入（n由最早需要的数据决定）。下面就可以分为两种情况了hhh，取决于真正比赛中test数据是怎么来的（因为是来自真实市场）。
            * <font style='color: red;'>假设每天都要更新，每天也要预测一次，即，每天都新加入这一天的数据</font>，这样在model中或者什么不知名的地方，我需要维护一个窗口，里面存着t-n到t的全部数据，并且在新来第t+1的数据时，要去掉头（t-n），加上尾巴（t+1）（很自然可以用Queue，就是一次enqueue一次dequeue，但我不知道对于dataframe有无对应的操作hhh）（当然可以不去掉头，一直添加，但内存会不会有问题我不确定）。
            * <font style='color: red;'>假设稍微仁慈一点hhh，test是全部给出的（攒足够多的天数再给）</font>，这样稍微好一些，只需要把训练集的尾巴和测试集拼起来，剩下的正常计算就好了。
        * 当然其实还有一种处理，大气一点hhh，test前n条不在乎了，由于之前数据的缺失那就NAN，这样肯定会影响预测效果，直到test中有足够多天数，再计算出完整的特征，然后 
          预测。

        * 按照上述讨论，我在提完特征再split是合理并且方便的hhh

* ......

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from scipy.spatial.distance import correlation

from IPython.display import display


In [2]:
# data path
path = './data'
p_train_price = path + '/train_files/stock_prices.csv'
p_train_trades = path + '/train_files/trades.csv'
p_stock_list = path + '/stock_list.csv'


In [4]:
def read_csv() -> tuple:
    '''
    read three csv
    '''
    df_price = pd.read_csv(p_train_price)
    df_trades = pd.read_csv(p_train_trades)
    df_list = pd.read_csv(p_stock_list)

    return (df_price, df_trades, df_list)


In [5]:
def check_report_na():
    '''
    we check nan for each stocks
    '''
    grouped = df_price.groupby(['SecuritiesCode'])

    rpt = np.empty(shape=(2000, 12), dtype='int64')

    i = 0
    for k, v in grouped:
        rpt[i] = v.isna().sum().values
        i += 1

    rpt = rpt.transpose()
    rpt = pd.DataFrame(rpt, index=['RowId', 'Date', 'SecuritiesCode', 'Open', 'High', 'Low', 'Close',
                                   'Volume', 'AdjustmentFactor', 'ExpectedDividend', 'SupervisionFlag',
                                   'Target'],
                       columns=list(grouped.groups.keys()))

    rpt['sum'] = rpt.sum(axis=1)
    display(rpt)

    # return rpt


In [6]:
def correct_missing_values(df):
    """
    Returns a DataFrame with no NaN values for OHLC columns    
    ref https://www.kaggle.com/code/vweimer/jpx-adjusted-close-and-volume-and-features/notebook?scriptVersionId=96081798
    """
    # 1: set volume values to NaN where volume is 0
    df['Volume'] = df['Volume'].replace(0, np.nan)

    # 2: forward-fill missing values
    df = df.fillna(df.groupby('SecuritiesCode').ffill())

    # 3: drop all rows that are left with NaN values, e.g., ticket 3540 has around 200 values that are NaN at beginning
    df = df.dropna(subset=['Volume'])

    # Check number of remaining rows
    print(f"Number of rows after filling missing values: {len(df.index)}")

    return df


In [7]:
class Adjust_Price_Volume(BaseEstimator, TransformerMixin):
    '''
    adjust price and volume ref https://www.kaggle.com/code/vweimer/jpx-adjusted-close-and-volume-and-features/notebook?scriptVersionId=96081798
    '''

    def __init__(self) -> None:
        super().__init__()
        self.df_fea_custom = pd.DataFrame()

    def fit(self, X, y=None):

        def generate_adjusted_close(df):

            # sort data to generate CumulativeAdjustmentFactor
            df = df.sort_values('Date', ascending=False)
            # generate CumulativeAdjustmentFactor
            df.loc[:, 'CumAdjustmentFactor'] = df['AdjustmentFactor'].cumprod()
            # generate AdjustedClose
            df.loc[:, 'adj.Open'] = (df['Open'] * df['CumAdjustmentFactor'])
            df.loc[:, 'adj.High'] = (df['High'] * df['CumAdjustmentFactor'])
            df.loc[:, 'adj.Low'] = (df['Low'] * df['CumAdjustmentFactor'])
            df.loc[:, 'adj.Close'] = (df['Close'] * df['CumAdjustmentFactor'])

            # reverse order - since adjusted volume needs to be in reverse AFTER the stock split
            df = df.sort_values('Date')

            # Generate cumulative adjustment factor volume
            df.loc[:, 'CumAdjustmentFactorVol'] = df['AdjustmentFactor'].cumprod()

            # generate AdjustedVolume
            df['adj.Volume'] = df['Volume'].where(
                df['AdjustmentFactor'] != 1, (df['Volume'] * df['CumAdjustmentFactorVol']).astype('int64'))

            return df

        self.df_fea_custom = X.groupby(
            'SecuritiesCode').apply(generate_adjusted_close)

        return self

    def transform(self, X):
        # print(X.shape)
        # print(self.df_fea_custom.shape)
        # print(self.df_fea_custom.index)
        self.df_fea_custom.sort_index(level=1, inplace=True)
        # print(self.df_fea_custom.index)
        self.df_fea_custom.index = range(self.df_fea_custom.shape[0])

        # Seriously destroy the encapsulation hhh， but for convenience
        global DDate
        DDate = pd.to_datetime(self.df_fea_custom['Date'])
        return self.df_fea_custom


In [8]:
class Feature_Sector_Index_OtherInfo(BaseEstimator, TransformerMixin):
    '''
    1 oridinal code of week, month\\
    2 one hot encode of section/Marketsegment\\
    3 within a day, we use volume weighted average price\\
        and average volume to encode 33sector, 17sector, newindex
    '''

    def __init__(self) -> None:
        super().__init__()
        self.df_fea_custom = pd.DataFrame()

    def fit(self, X, y=None):
        # print(X.shape)
        # print(self.df_fea_custom.shape)

        # 1 oridinal code of week, month
        temp = pd.to_datetime(X['Date'])
        self.df_fea_custom['Month'] = temp.dt.month
        self.df_fea_custom['Week'] = temp.dt.weekday

        # merge accroding to stock_list
        merge_cols = ['SecuritiesCode', 'Section/Products', 'NewMarketSegment',
                      '33SectorCode', '17SectorCode', 'NewIndexSeriesSizeCode', 'Universe0']

        merged = pd.merge(
            X, df_list.loc[:, merge_cols], on='SecuritiesCode', how='left')

        # 2 one hot encoding of these two
        for ss in ['Section/Products', 'NewMarketSegment']:
            temp = pd.get_dummies(merged[ss], drop_first=True)
            self.df_fea_custom[temp.columns] = temp

        # 3 within a day encode 33sector, 17sector, newindex
        # def apply func
        def encoding_3(group: pd.DataFrame, by: str):
            p = group['adj.Close']
            w = group['adj.Volume']
            #p = group['Close']
            #w = group['Volume']

            # here this kind of weighted average, I'm not sure if it's reasonable --- need professional knowledge
            group[(by+'_wp')] = (p*w).sum()/w.sum()
            group[(by+'_vol')] = w.mean()

            return group

        # encoding
        merged_mini = merged[['Date', '33SectorCode', '17SectorCode',
                              'NewIndexSeriesSizeCode', 'adj.Close', 'adj.Volume']].copy()

        for by in ['33SectorCode', '17SectorCode', 'NewIndexSeriesSizeCode']:
            merged_mini = merged_mini.groupby(
                ['Date', by]).apply(encoding_3, by=by)

        self.df_fea_custom[['33SectorCode_wp', '33SectorCode_vol',
                            '17SectorCode_wp', '17SectorCode_vol',
                            'NewIndexSeriesSizeCode_wp', 'NewIndexSeriesSizeCode_vol'
                            ]] = merged_mini.iloc[:, -6:]

        return self

    def transform(self, X):
        # print(X.shape)
        # print(self.df_fea_custom.shape)
        return self.df_fea_custom


In [9]:
class Feature_Raw(BaseEstimator, TransformerMixin):
    '''
    5 raw transaction data 'Open', 'High', 'Low', 'Close', 'Volume'
    '''

    def __init__(self) -> None:
        super().__init__()
        self.df_fea_custom = pd.DataFrame()

    def fit(self, X, y=None):
        # print(X.shape)
        # print(self.df_fea_custom.shape)
        self.df_fea_custom = X[
            ['adj.Open', 'adj.High', 'adj.Low', 'adj.Close', 'adj.Volume']
        ].copy()

        return self

    def transform(self, X):
        # print(X.shape)
        # print(self.df_fea_custom.shape)

        return self.df_fea_custom


In [10]:
class Feature_Custom_v1(BaseEstimator, TransformerMixin):
    '''
    some intuitive features
    '''

    def __init__(self) -> None:
        super().__init__()
        self.df_fea_custom = pd.DataFrame()

    def fit(self, X, y=None):
        # print(X.shape)
        # print(self.df_fea_custom.shape)
        self.df_fea_custom['adj.Close/Open'] = X['adj.Close'] / X['adj.Open']
        self.df_fea_custom['adj.Close-Open'] = X['adj.Close'] - X['adj.Open']
        self.df_fea_custom['adj.High-Low'] = X['adj.High'] - X['adj.Low']
        self.df_fea_custom['adj.High/Low'] = X['adj.High'] / X['adj.Low']

        self.df_fea_custom['adj.Mean'] = X[['adj.Open',
                                            'adj.High', 'adj.Low', 'adj.Close']].mean(axis=1)
        self.df_fea_custom['adj.High/Mean'] = X['adj.High'] / \
            self.df_fea_custom['adj.Mean']
        self.df_fea_custom['adj.Low/Mean'] = X['adj.Low'] / \
            self.df_fea_custom['adj.Mean']

        self.df_fea_custom['Upper_Shadow'] = X['adj.High'] - np.maximum(
            X['adj.Close'], X['adj.Open']
        )
        self.df_fea_custom['Lower_Shadow'] = (
            np.minimum(X['adj.Close'], X['adj.Open']) - X['adj.Low']
        )

        return self

    def transform(self, X):
        # print(X.shape)
        # print(self.df_fea_custom.shape)
        return self.df_fea_custom


In [11]:
class Feature_(BaseEstimator, TransformerMixin):
    ...


In [12]:
class Get_Target(BaseEstimator, TransformerMixin):
    '''
    target
    '''

    def __init__(self) -> None:
        super().__init__()
        #self.df_fea_custom = pd.DataFrame()

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        #self.df_fea_custom = X['Target'].copy()
        # return self.df_fea_custom
        return pd.DataFrame(X['Target'])


In [13]:
class To_df_get_Xy(BaseEstimator, TransformerMixin):
    '''
    get X_y and split
    '''

    def __init__(self) -> None:
        super().__init__()
        self.df_fea_custom = None

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        self.df_fea_custom = pd.DataFrame(X)
        self.df_fea_custom.reset_index(drop=True)

        # here xx is dataframe y is series (can be modified according to model)
        xx = self.df_fea_custom.iloc[:, 0:-1]
        y = self.df_fea_custom.iloc[:, -1]
        #xx = self.df_fea_custom.values[:, 0:-1]
        #y = self.df_fea_custom.values[:, -1]

        return xx, y


In [14]:
class Print(BaseEstimator, TransformerMixin):
    '''
    print
    '''

    def __init__(self) -> None:
        super().__init__()

    def fit(self, X: pd.DataFrame, y=None):
        display(X)
        print(X.shape)
        # print(X.columns)
        return self

    def transform(self, X):
        return X


In [15]:
feaunion_total = FeatureUnion([
    ('raw_features', Feature_Raw()),
    ('intuitive_features', Feature_Custom_v1()),
    #('finance_features', Feature_Finance_Factors()),
    #('lag_features', Feature_Lag()),
    ('encodings', Feature_Sector_Index_OtherInfo()),
    ('get_target', Get_Target()),
])


In [16]:
pipe_FE = Pipeline([
    ('adjust_price_volume', Adjust_Price_Volume()),
    ('FE', feaunion_total),
    ('get_Xy', To_df_get_Xy()),
])


In [17]:
% % time
# read csv
df_price, df_trades, df_list = read_csv()
DDate = pd.Series(dtype='datetime64[ns]')


Wall time: 2.88 s


In [18]:
% % time
# check na -> fillna -> check na
# ExpectedDividend is left since I don't how to use it...
check_report_na()
df_price = correct_missing_values(df_price)
check_report_na()


,1301,1332,1333,1375,1376,1377,1379,1381,1407,1413,...,9983,9984,9987,9989,9990,9991,9993,9994,9997,sum
RowId,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Date,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SecuritiesCode,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Open,1,1,1,1,1,1,1,3,1,1,...,1,1,1,1,1,1,1,1,1,7608
High,1,1,1,1,1,1,1,3,1,1,...,1,1,1,1,1,1,1,1,1,7608
Low,1,1,1,1,1,1,1,3,1,1,...,1,1,1,1,1,1,1,1,1,7608
Close,1,1,1,1,1,1,1,3,1,1,...,1,1,1,1,1,1,1,1,1,7608
Volume,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AdjustmentFactor,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ExpectedDividend,1197,1192,1197,294,1192,1192,1192,1193,1192,1140,...,1192,1192,1192,1192,1192,1192,1192,1192,1192,2313666


Number of rows after filling missing values: 2332267


,1301,1332,1333,1375,1376,1377,1379,1381,1407,1413,...,9983,9984,9987,9989,9990,9991,9993,9994,9997,sum
RowId,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Date,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SecuritiesCode,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Open,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
High,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Low,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Close,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Volume,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AdjustmentFactor,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ExpectedDividend,56,56,56,4,96,96,56,118,34,65,...,34,56,56,56,56,56,34,56,56,127215


Wall time: 8.58 s


In [19]:
% % time
xx, y = pipe_FE.fit_transform(df_price)

print(xx.shape)
print(y.shape)


(2332267, 28)
(2332267,)
Wall time: 1min 30s


0~4 'raw_features' x5  
5~13 'intuitive_features' x9  
14,15 month week ordinal x2  
16~21 'Section/Products', 'NewMarketSegment' 4 onehot + 2 onehot x6  
22~27 ['33SectorCode_wp', '33SectorCode_vol','17SectorCode_wp', '17SectorCode_vol', 'NewIndexSeriesSizeCode_wp', 'NewIndexSeriesSizeCode_vol'] x6  
5+9+2+6+6=28

In [20]:
# check nan
np.array(xx.isna().sum())


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0], dtype=int64)

In [21]:
display(xx.iloc[-2:-1, 0:5])
display(xx.iloc[-2:-1, 5:14])
display(xx.iloc[-2:-1, 14:16])
display(xx.iloc[-2:-1, 16:22])
display(xx.iloc[-2:-1, 22:28])


,0,1,2,3,4
2332265,2388.0,2396.0,2380.0,2389.0,6500.0


,5,6,7,8,9,10,11,12,13
2332265,1.000419,1.0,16.0,1.006723,2388.25,1.003245,0.996546,7.0,8.0


,14,15
2332265,12.0,4.0


,16,17,18,19,20,21
2332265,0.0,0.0,0.0,0.0,0.0,1.0


,22,23,24,25,26,27
2332265,2990.427528,270550.793296,2990.427528,270550.793296,1320.092568,121288.56768


In [22]:
xx['Date'] = DDate


In [23]:
xx


,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,Date
0,2734.0,2755.0,2730.0,2742.0,31400.0,1.002926,8.0,25.0,1.009158,2740.25,...,0.0,1.0,0.0,970.009936,477342.857143,2944.824415,404303.260870,1437.794643,1.588192e+05,2017-01-04
1,568.0,576.0,563.0,571.0,2798500.0,1.005282,3.0,13.0,1.023091,569.50,...,0.0,1.0,0.0,970.009936,477342.857143,2944.824415,404303.260870,2327.637408,2.165597e+06,2017-01-04
2,3150.0,3210.0,3140.0,3210.0,270800.0,1.019048,60.0,70.0,1.022293,3177.50,...,0.0,1.0,0.0,970.009936,477342.857143,2944.824415,404303.260870,2327.637408,2.165597e+06,2017-01-04
3,1510.0,1550.0,1510.0,1550.0,11300.0,1.026490,40.0,40.0,1.026490,1530.00,...,0.0,0.0,1.0,970.009936,477342.857143,2944.824415,404303.260870,1437.794643,1.588192e+05,2017-01-04
4,3270.0,3350.0,3270.0,3330.0,150800.0,1.018349,60.0,80.0,1.024465,3305.00,...,0.0,1.0,0.0,970.009936,477342.857143,2944.824415,404303.260870,1554.451560,5.079538e+05,2017-01-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2332262,514.0,528.0,513.0,528.0,44200.0,1.027237,14.0,15.0,1.029240,520.75,...,0.0,1.0,0.0,2990.427528,270550.793296,2990.427528,270550.793296,1320.092568,1.212886e+05,2021-12-03
2332263,782.0,794.0,782.0,794.0,35900.0,1.015345,12.0,12.0,1.015345,788.00,...,0.0,1.0,0.0,2225.923640,301972.748344,2225.923640,301972.748344,1320.092568,1.212886e+05,2021-12-03
2332264,1690.0,1690.0,1645.0,1645.0,7200.0,0.973373,-45.0,45.0,1.027356,1667.50,...,0.0,0.0,1.0,2990.427528,270550.793296,2990.427528,270550.793296,1320.092568,1.212886e+05,2021-12-03
2332265,2388.0,2396.0,2380.0,2389.0,6500.0,1.000419,1.0,16.0,1.006723,2388.25,...,0.0,0.0,1.0,2990.427528,270550.793296,2990.427528,270550.793296,1320.092568,1.212886e+05,2021-12-03


### split cv accroding to Date...( ref https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html)

In [24]:
df_price


,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400.0,1.0,NaN,False,0.000730
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500.0,1.0,NaN,False,0.012324
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800.0,1.0,NaN,False,0.006154
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300.0,1.0,NaN,False,0.011053
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800.0,1.0,NaN,False,0.003026
...,...,...,...,...,...,...,...,...,...,...,...,...
2332526,20211203_9990,2021-12-03,9990,514.0,528.0,513.0,528.0,44200.0,1.0,0.0,False,0.034816
2332527,20211203_9991,2021-12-03,9991,782.0,794.0,782.0,794.0,35900.0,1.0,15.0,False,0.025478
2332528,20211203_9993,2021-12-03,9993,1690.0,1690.0,1645.0,1645.0,7200.0,1.0,13.5,False,-0.004302
2332529,20211203_9994,2021-12-03,9994,2388.0,2396.0,2380.0,2389.0,6500.0,1.0,24.0,False,0.009098
